
#DAY10
1.把之前所有的处理手段都处理一遍，回顾一下全流程，以后就用处理好的部分直接完成
2.开始机器学习建模（简单建模，不涉及调参）和评估
## 预处理流程回顾
1. 导入库
2. 读取数据查看数据信息--理解数据
3. 缺失值处理
4. 异常值处理
5. 离散值处理
6. 删除无用列
7. 划分数据集
8. 特征工程
9. 模型训练
10. 模型评估
11. 模型保存
12. 模型预测
#### 导入所需要的包
这里其实是写完后一起整理到这里的


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
data = pd.read_csv('data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Id                            7500 non-null   int64  
 1   Home Ownership                7500 non-null   object 
 2   Annual Income                 5943 non-null   float64
 3   Years in current job          7129 non-null   object 
 4   Tax Liens                     7500 non-null   float64
 5   Number of Open Accounts       7500 non-null   float64
 6   Years of Credit History       7500 non-null   float64
 7   Maximum Open Credit           7500 non-null   float64
 8   Number of Credit Problems     7500 non-null   float64
 9   Months since last delinquent  3419 non-null   float64
 10  Bankruptcies                  7486 non-null   float64
 11  Purpose                       7500 non-null   object 
 12  Term                          7500 non-null   object 
 13  Cur

,Id,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


以下是对该数据集进行预处理的步骤及顺序：

一共有17个特征，分别处理

#### 1.缺失值处理
* Annual Income：有5943个非空值，存在缺失值。可以考虑使用均值填充、中位数填充或者基于其他相关特征进行回归预测填充。例如，如果“Home Ownership”和“Annual Income”有一定相关性，可根据不同房屋所有权类型的平均收入来填充缺失值。
* Years in current job：7129个非空值，存在缺失值。由于是对象类型，可能需要先将其转换为合适的数值类型再进行处理。比如将“10+ years”转换为10，“8 years”转换为8等，然后再用众数或中位数填充缺失值。
* Months since last delinquent：只有3419个非空值，缺失值较多。若该特征对目标变量影响较大，可尝试用多重填补法等较为复杂的方法进行填充；若影响较小，也可直接删除含有缺失值的行，但要注意可能会导致数据量损失较大。
* Credit Score：5943个非空值，存在缺失值。可参照“Annual Income”的处理方式，根据与其他特征的相关性来选择合适的填充方法。
#### 4.数据类型转换
* Years in current job：将其从对象类型转换为数值类型，方便后续的计算和模型处理。
* Home Ownership、Purpose、Term：这些对象类型的特征可以进行独热编码或标签编码。如果特征的类别数较少且没有明显的顺序关系，独热编码较为合适；如果有一定的顺序关系，如“Term”的“Short Term”和“Long Term”，可以考虑标签编码。
#### 3.异常值处理
* 对于数值型特征，如“Annual Income”“Current Loan Amount”等，可以通过箱线图等方法检测异常值。如果存在异常值，需根据实际情况决定是否进行处理。若是数据录入错误等原因导致的异常值，可以进行修正或删除；若是真实存在的极端值，可能需要保留，但在某些模型中可能需要进行特殊处理，如采用稳健的统计方法或对数据进行变换。
#### 4.特征缩放
* 对数值型特征进行特征缩放，将其缩放到相同的尺度，以避免某些特征因数值较大而在模型中占据主导地位。常用的方法有Min - Max标准化和Z - score标准化。例如，“Annual Income”“Years of Credit History”“Credit Score”等特征的取值范围差异较大，可通过特征缩放将它们的取值范围统一到[0, 1]或均值为0、标准差为1的分布上。
#### 5.特征工程
* 衍生新特征：根据已有特征创建新的特征，可能会对模型性能有提升。例如，可以计算“Debt - to - Income Ratio”（负债收入比），即“Monthly Debt”与“Annual Income”的比值，来反映客户的债务负担情况。
* 特征选择：通过相关性分析等方法，选择与目标变量“Credit Default”相关性较高的特征，去除相关性较低或冗余的特征，以降低模型的复杂度和过拟合的风险。
在实际操作中，需要先进行缺失值处理，然后进行数据类型转换，接着处理异常值，再进行特征缩放，最后进行特征工程。这样的顺序可以保证数据在预处理过程中的一致性和有效性，为后续的机器学习模型训练提供高质量的数据。

### 首先处理object对象
因为最后的输入都是数值类型，所以先给字符串变量处理了

In [3]:
# 先筛选字符串变量 
discrete_features = data.select_dtypes(include=['object']).columns.tolist()
discrete_features

['Home Ownership', 'Years in current job', 'Purpose', 'Term']

In [4]:
# 依次查看内容
for feature in discrete_features:
    print(f"\n{feature}的取值：")
    print(data[feature].value_counts())


Home Ownership的取值：
Home Ownership
Home Mortgage    3637
Rent             3204
Own Home          647
Have Mortgage      12
Name: count, dtype: int64

Years in current job的取值：
Years in current job
10+ years    2332
2 years       705
3 years       620
< 1 year      563
5 years       516
1 year        504
4 years       469
6 years       426
7 years       396
8 years       339
9 years       259
Name: count, dtype: int64

Purpose的取值：
Purpose
debt consolidation      5944
other                    665
home improvements        412
business loan            129
buy a car                 96
medical bills             71
major purchase            40
take a trip               37
buy house                 34
small business            26
wedding                   15
moving                    11
educational expenses      10
vacation                   8
renewable energy           2
Name: count, dtype: int64

Term的取值：
Term
Short Term    5556
Long Term     1944
Name: count, dtype: int64


Home Ownership需要标签编码。
* 住房抵押贷款：3637 这个是有房贷，有房子
* 租房：3204 没房子
* 拥有自有住房：647 这个没贷款，有房子
* 有贷款：12 这个是有其他贷款，有房子，没房贷
* 名称：房屋所有权，数据类型：int64
按照贷款严重程度（抗风险能力），依次是：自有住房 ＜ 租房 ＜ 有其他贷款 ＜ 住房抵押贷款
所以按照这个逻辑来进行编码
Years in current job 做标签编码
Purpose做独热编码
Term直接做0-1映射即可，二分类问题，处理后更改自己的名字为1的类别

#### 本质上和上一篇博客一样，都是用map映射

In [5]:
# Home Ownership 标签编码 
home_ownership_mapping = {
    'Own Home': 1,
    'Rent': 2,
    'Have Mortgage': 3,
    'Home Mortgage': 4
}
data['Home Ownership'] = data['Home Ownership'].map(home_ownership_mapping)

# Years in current job 标签编码
years_in_job_mapping = {
    '< 1 year': 1,
    '1 year': 2,
    '2 years': 3,
    '3 years': 4,
    '4 years': 5,
    '5 years': 6,
    '6 years': 7,
    '7 years': 8,
    '8 years': 9,
    '9 years': 10,
    '10+ years': 11
}
data['Years in current job'] = data['Years in current job'].map(years_in_job_mapping)

# Purpose 独热编码，记得需要将bool类型转换为数值
data = pd.get_dummies(data, columns=['Purpose'])
data2 = pd.read_csv("data.csv") # 重新读取数据，用来做列名对比
list_final = [] # 新建一个空列表，用于存放独热编码后新增的特征名
for i in data.columns:
    if i not in data2.columns:
       list_final.append(i) # 这里打印出来的就是独热编码后的特征名
for i in list_final:
    data[i] = data[i].astype(int) # 这里的i就是独热编码后的特征名



# Term 0 - 1 映射
term_mapping = {
    'Short Term': 0,
    'Long Term': 1
}
data['Term'] = data['Term'].map(term_mapping)
data.rename(columns={'Term': 'Long Term'}, inplace=True) # 重命名列

### 处理数值型对象
缺失值填补

In [6]:
continuous_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()  #把筛选出来的列名转换成列表
 # 连续特征用中位数补全
for feature in continuous_features:     
    mode_value = data[feature].mode()[0]            #获取该列的众数。
    data[feature].fillna(mode_value, inplace=True)          #用众数填充该列的缺失值，inplace=True表示直接在原数据上修改。

#### 异常值处理
异常值一般不处理，或者结合对照试验处理和不处理都尝试下，但是论文中要写这个，作为个工作量

In [7]:
data.info()  #查看数据基本信息
#会发现所有的列都是数值类型，没有缺失值。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Id                            7500 non-null   int64  
 1   Home Ownership                7500 non-null   int64  
 2   Annual Income                 7500 non-null   float64
 3   Years in current job          7500 non-null   float64
 4   Tax Liens                     7500 non-null   float64
 5   Number of Open Accounts       7500 non-null   float64
 6   Years of Credit History       7500 non-null   float64
 7   Maximum Open Credit           7500 non-null   float64
 8   Number of Credit Problems     7500 non-null   float64
 9   Months since last delinquent  7500 non-null   float64
 10  Bankruptcies                  7500 non-null   float64
 11  Long Term                     7500 non-null   int64  
 12  Current Loan Amount           7500 non-null   float64
 13  Cur

## 机器学习模型建模
首先进行数据划分，记住之前课上的划分方法：只要调参就要考2次

这里我们先不调参，所以只需要划分一次即可

首先声明一点，这些模型本文不深究模型的原理，因为现在作者也不会，我只知道模型的使用方法，所以本文只是使用这些模型，如果后期会学习模型原理的话，将会在之后的博客里面更新。
#### 数据划分

In [8]:
# 划分训练集和测试机
from sklearn.model_selection import train_test_split
X = data.drop(['Credit Default'], axis=1)  # 特征，axis=1表示按列删除
y = data['Credit Default']  # 标签
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 划分数据集，20%作为测试集，随机种子为42
# 训练集和测试集的形状
print(f"训练集形状: {X_train.shape}, 测试集形状: {X_test.shape}")  # 打印训练集和测试集的形状

训练集形状: (6000, 31), 测试集形状: (1500, 31)


In [10]:
from sklearn.svm import SVC #支持向量机分类器
from sklearn.neighbors import KNeighborsClassifier #K近邻分类器
from sklearn.linear_model import LogisticRegression #逻辑回归分类器
import xgboost as xgb #XGBoost分类器
import lightgbm as lgb #LightGBM分类器
from sklearn.ensemble import RandomForestClassifier #随机森林分类器
from catboost import CatBoostClassifier #CatBoost分类器
from sklearn.tree import DecisionTreeClassifier #决策树分类器
from sklearn.naive_bayes import GaussianNB #高斯朴素贝叶斯分类器
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 用于评估分类器性能的指标
from sklearn.metrics import classification_report, confusion_matrix #用于生成分类报告和混淆矩阵
import warnings #用于忽略警告信息
warnings.filterwarnings("ignore") # 忽略所有警告信息

In [11]:
# SVM
svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

print("\nSVM 分类报告：")
print(classification_report(y_test, svm_pred))  # 打印分类报告
print("SVM 混淆矩阵：")
print(confusion_matrix(y_test, svm_pred))  # 打印混淆矩阵

# 计算 SVM 评估指标，这些指标默认计算正类的性能
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred)
svm_recall = recall_score(y_test, svm_pred)
svm_f1 = f1_score(y_test, svm_pred)
print("SVM 模型评估指标：")
print(f"准确率: {svm_accuracy:.4f}")
print(f"精确率: {svm_precision:.4f}")
print(f"召回率: {svm_recall:.4f}")
print(f"F1 值: {svm_f1:.4f}")


SVM 分类报告：
              precision    recall  f1-score   support

           0       0.71      1.00      0.83      1059
           1       0.00      0.00      0.00       441

    accuracy                           0.71      1500
   macro avg       0.35      0.50      0.41      1500
weighted avg       0.50      0.71      0.58      1500

SVM 混淆矩阵：
[[1059    0]
 [ 441    0]]
SVM 模型评估指标：
准确率: 0.7060
精确率: 0.0000
召回率: 0.0000
F1 值: 0.0000


classification_report它会生成所有类别的指标

准确率（Accuracy）是一个全局指标，衡量所有类别预测正确的比例 (TP + TN) / (TP + TN + FP + FN)。它不区分正负类，所以它只有一个值，不区分类别

单独调用的 precision_score, recall_score, f1_score 在二分类中默认只计算正类（标签 1）的性能。由于模型从未成功预测出类别 1（TP=0），所以这些指标对类别 1 来说都是 0。

In [12]:
# KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)

print("\nKNN 分类报告：")
print(classification_report(y_test, knn_pred))
print("KNN 混淆矩阵：")
print(confusion_matrix(y_test, knn_pred))

knn_accuracy = accuracy_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred)
knn_recall = recall_score(y_test, knn_pred)
knn_f1 = f1_score(y_test, knn_pred)
print("KNN 模型评估指标：")
print(f"准确率: {knn_accuracy:.4f}")
print(f"精确率: {knn_precision:.4f}")
print(f"召回率: {knn_recall:.4f}")
print(f"F1 值: {knn_f1:.4f}")


KNN 分类报告：
              precision    recall  f1-score   support

           0       0.73      0.86      0.79      1059
           1       0.41      0.24      0.30       441

    accuracy                           0.68      1500
   macro avg       0.57      0.55      0.54      1500
weighted avg       0.64      0.68      0.65      1500

KNN 混淆矩阵：
[[908 151]
 [336 105]]
KNN 模型评估指标：
准确率: 0.6753
精确率: 0.4102
召回率: 0.2381
F1 值: 0.3013


In [14]:
# 逻辑回归
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train, y_train)
logreg_pred = logreg_model.predict(X_test)

print("\n逻辑回归 分类报告：")
print(classification_report(y_test, logreg_pred))
print("逻辑回归 混淆矩阵：")
print(confusion_matrix(y_test, logreg_pred))

logreg_accuracy = accuracy_score(y_test, logreg_pred)
logreg_precision = precision_score(y_test, logreg_pred)
logreg_recall = recall_score(y_test, logreg_pred)
logreg_f1 = f1_score(y_test, logreg_pred)
print("逻辑回归 模型评估指标：")
print(f"准确率: {logreg_accuracy:.4f}")
print(f"精确率: {logreg_precision:.4f}")
print(f"召回率: {logreg_recall:.4f}")
print(f"F1 值: {logreg_f1:.4f}")


逻辑回归 分类报告：
              precision    recall  f1-score   support

           0       0.75      0.99      0.85      1059
           1       0.86      0.20      0.33       441

    accuracy                           0.76      1500
   macro avg       0.80      0.59      0.59      1500
weighted avg       0.78      0.76      0.70      1500

逻辑回归 混淆矩阵：
[[1044   15]
 [ 351   90]]
逻辑回归 模型评估指标：
准确率: 0.7560
精确率: 0.8571
召回率: 0.2041
F1 值: 0.3297


In [15]:
# 朴素贝叶斯
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

print("\n朴素贝叶斯 分类报告：")
print(classification_report(y_test, nb_pred))
print("朴素贝叶斯 混淆矩阵：")
print(confusion_matrix(y_test, nb_pred))

nb_accuracy = accuracy_score(y_test, nb_pred)
nb_precision = precision_score(y_test, nb_pred)
nb_recall = recall_score(y_test, nb_pred)
nb_f1 = f1_score(y_test, nb_pred)
print("朴素贝叶斯 模型评估指标：")
print(f"准确率: {nb_accuracy:.4f}")
print(f"精确率: {nb_precision:.4f}")
print(f"召回率: {nb_recall:.4f}")
print(f"F1 值: {nb_f1:.4f}")
    


朴素贝叶斯 分类报告：
              precision    recall  f1-score   support

           0       0.98      0.19      0.32      1059
           1       0.34      0.99      0.50       441

    accuracy                           0.43      1500
   macro avg       0.66      0.59      0.41      1500
weighted avg       0.79      0.43      0.38      1500

朴素贝叶斯 混淆矩阵：
[[204 855]
 [  5 436]]
朴素贝叶斯 模型评估指标：
准确率: 0.4267
精确率: 0.3377
召回率: 0.9887
F1 值: 0.5035


In [16]:
# 随机森林
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print("\n随机森林 分类报告：")
print(classification_report(y_test, rf_pred))
print("随机森林 混淆矩阵：")
print(confusion_matrix(y_test, rf_pred))

rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)
print("随机森林 模型评估指标：")
print(f"准确率: {rf_accuracy:.4f}")
print(f"精确率: {rf_precision:.4f}")
print(f"召回率: {rf_recall:.4f}")
print(f"F1 值: {rf_f1:.4f}")


随机森林 分类报告：
              precision    recall  f1-score   support

           0       0.77      0.97      0.86      1059
           1       0.79      0.30      0.43       441

    accuracy                           0.77      1500
   macro avg       0.78      0.63      0.64      1500
weighted avg       0.77      0.77      0.73      1500

随机森林 混淆矩阵：
[[1023   36]
 [ 309  132]]
随机森林 模型评估指标：
准确率: 0.7700
精确率: 0.7857
召回率: 0.2993
F1 值: 0.4335


In [17]:
# XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

print("\nXGBoost 分类报告：")
print(classification_report(y_test, xgb_pred))
print("XGBoost 混淆矩阵：")
print(confusion_matrix(y_test, xgb_pred))

xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_precision = precision_score(y_test, xgb_pred)
xgb_recall = recall_score(y_test, xgb_pred)
xgb_f1 = f1_score(y_test, xgb_pred)
print("XGBoost 模型评估指标：")
print(f"准确率: {xgb_accuracy:.4f}")
print(f"精确率: {xgb_precision:.4f}")
print(f"召回率: {xgb_recall:.4f}")
print(f"F1 值: {xgb_f1:.4f}")


XGBoost 分类报告：
              precision    recall  f1-score   support

           0       0.77      0.91      0.84      1059
           1       0.62      0.37      0.46       441

    accuracy                           0.75      1500
   macro avg       0.70      0.64      0.65      1500
weighted avg       0.73      0.75      0.72      1500

XGBoost 混淆矩阵：
[[960  99]
 [280 161]]
XGBoost 模型评估指标：
准确率: 0.7473
精确率: 0.6192
召回率: 0.3651
F1 值: 0.4593


In [18]:
# LightGBM
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)

print("\nLightGBM 分类报告：")
print(classification_report(y_test, lgb_pred))
print("LightGBM 混淆矩阵：")
print(confusion_matrix(y_test, lgb_pred))

lgb_accuracy = accuracy_score(y_test, lgb_pred)
lgb_precision = precision_score(y_test, lgb_pred)
lgb_recall = recall_score(y_test, lgb_pred)
lgb_f1 = f1_score(y_test, lgb_pred)
print("LightGBM 模型评估指标：")
print(f"准确率: {lgb_accuracy:.4f}")
print(f"精确率: {lgb_precision:.4f}")
print(f"召回率: {lgb_recall:.4f}")
print(f"F1 值: {lgb_f1:.4f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1672, number of negative: 4328
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2160
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.278667 -> initscore=-0.951085
[LightGBM] [Info] Start training from score -0.951085

LightGBM 分类报告：
              precision    recall  f1-score   support

           0       0.78      0.94      0.85      1059
           1       0.70      0.36      0.47       441

    accuracy                           0.77      1500
   macro avg       0.74      0.65      0.66      1500
weighted avg       0.75      0.77      0.74      1500

LightGBM 混淆矩阵：
[[992  

### 可以看下这些模型的性能对比
| 模型名称 | 准确率 | 精确率（正类） | 召回率（正类） | F1值（正类） | 精确率（负类） | 召回率（负类） | F1值（负类） |
| --- | --- | --- | --- | --- | --- | --- | --- |
| SVM | 0.7060 | 0.0000 | 0.0000 | 0.0000 | 0.71 | 1.00 | 0.83 |
| KNN | 0.6753 | 0.4102 | 0.2381 | 0.3013 | 0.73 | 0.86 | 0.79 |
| 逻辑回归 | 0.7560 | 0.8571 | 0.2041 | 0.3297 | 0.75 | 0.99 | 0.85 |
| 朴素贝叶斯 | 0.4267 | 0.3377 | 0.9887 | 0.5035 | 0.98 | 0.19 | 0.32 |
| 决策树 | 0.6773 | 0.4564 | 0.5102 | 0.4818 | 0.79 | 0.75 | 0.77 |
| 随机森林 | 0.7700 | 0.7857 | 0.2993 | 0.4335 | 0.77 | 0.97 | 0.86 |
| XGBoost | 0.7473 | 0.6192 | 0.3651 | 0.4593 | 0.77 | 0.91 | 0.84 |
| LightGBM | 0.7660 | 0.7009 | 0.3560 | 0.4722 | 0.78 | 0.94 | 0.85 |


可以看到随机森林、LightGBM和XGBoost的效果都比较好，所以我们最终做实验的时候可以选择随机森林作为我们的模型。